In [1]:
import numpy as np

In [2]:
# load files
with open('aminoacids.txt') as f:
    aminos = f.read().split()
amino1 = aminos[0]
amino2 = aminos[1]    

alphabet = np.array(['a','c','t','g','-'], dtype=str)
distance_matrix = np.genfromtxt('distance_matrix.txt', dtype=int, delimiter=',')
similarity_matrix = np.genfromtxt('similarity_matrix.txt', dtype=int, delimiter=',')

In [3]:
# distance calculation
def d(mode, a,b):
    if mode == 'distance':
        dist = distance_matrix[np.where(alphabet==a),np.where(alphabet==b)]
    
    if mode == 'similarity':
        dist = similarity_matrix[np.where(alphabet==a),np.where(alphabet==b)]
    
    return dist[0]

In [4]:
def preparation(amino1, amino2, edit_m, direction_m, distance_mode):
    # first column
    direction_m[1:,0,2] = 1

    for idx, i in enumerate('-'+amino1):
        if idx>0:     
            edit_m[idx,0] = d(distance_mode,i,'-') + edit_m[idx-1,0]  
            
    # first row
    direction_m[0,1:,1] = 1

    for idx, i in enumerate('-'+amino2):
        if idx>0:
            edit_m[0,idx] = d(distance_mode,'-',i) + edit_m[0,idx-1] 
        
    # internal cells
    for i, chr1 in enumerate(amino1, start=1):
        for j, chr2 in enumerate(amino2, start=1):
            edit_m[i,j] = from_where(distance_mode,direction_m,i,j,
                                     edit_m[i-1,j-1] + d(distance_mode,chr1,chr2),
                                     edit_m[i,j-1] + d(distance_mode,'-',chr2),
                                     edit_m[i-1,j] + d(distance_mode,chr1,'-')) 
    return edit_m, direction_m

In [5]:
def from_where(mode,direction_m,i,j,a,b,c):
    if mode == 'distance':
        val = min(a,b,c)
    
    if mode == 'similarity':
        val = max(a,b,c)
          
    # from the diagonal
    if val == a:
        direction_m[i,j,0] = 1
    
    # from the left
    if val == b:
        direction_m[i,j,1] = 1
    
    # from the upside
    if val == c:
        direction_m[i,j,2] = 1
        
    return val

In [6]:
def optimal_alignment(direction_m,i,j,str1,str2):  
    if direction_m[i,j,0] == 1:
        str1.insert(0,('-'+amino1)[i])
        str2.insert(0,('-'+amino2)[j])
        i = i - 1
        j = j - 1
        
    elif direction_m[i,j,1] == 1:
        str1.insert(0,'-')
        str2.insert(0,('-'+amino2)[j])
        j = j - 1
        
    elif direction_m[i,j,2] == 1:
        str1.insert(0,('-'+amino1)[i])
        str2.insert(0,'-')
        i = i - 1
        
    print("i:", i, "j:",j)
    
    if i == 0 and j == 0:
        return str1, str2
    
    return optimal_alignment(direction_m,i,j,str1,str2)    

In [7]:
# initialize matrices
edit_m = np.zeros((len(amino1)+1,len(amino2)+1), dtype=int)
direction_m = np.zeros((len(amino1)+1,len(amino2)+1,3), dtype=int)

In [8]:
edit_m,direction_m = preparation(amino1,amino2,edit_m,direction_m,'distance')

In [9]:
optimal_alignment(direction_m,len(amino1),len(amino2),[],[])

i: 5 j: 4
i: 4 j: 3
i: 3 j: 2
i: 2 j: 1
i: 1 j: 0
i: 0 j: 0


(['a', 'a', 'a', 'c', 'c', 'c'], ['-', 'c', 'c', 'c', 't', 't'])

In [10]:
# SIMILARITY APPROACH

In [11]:
# initialize matrices
sim_m = np.zeros((len(amino1)+1,len(amino2)+1), dtype=int)
sim_route = np.zeros((len(amino1)+1,len(amino2)+1,3), dtype=int)

In [12]:
sim_m, sim_route = preparation(amino1,amino2,sim_m,sim_route,'similarity')
print(sim_m)
optimal_alignment(sim_route,len(amino1),len(amino2),[],[])

[[ 0 -1 -2 -3 -4 -5]
 [-1  0 -1 -2 -3 -4]
 [-2 -1  0 -1 -2 -3]
 [-3 -2 -1  0 -1 -2]
 [-4  0  1  2  1  0]
 [-5 -1  3  4  3  2]
 [-6 -2  2  6  5  4]]
i: 6 j: 4
i: 6 j: 3
i: 5 j: 2
i: 4 j: 1
i: 3 j: 0
i: 2 j: 0
i: 1 j: 0
i: 0 j: 0


(['a', 'a', 'a', 'c', 'c', 'c', '-', '-'],
 ['-', '-', '-', 'c', 'c', 'c', 't', 't'])

In [13]:
# local alignment

In [14]:
def local_preparation(amino1, amino2, edit_m, direction_m, distance_mode):
    # internal cells
    for i, chr1 in enumerate(amino1, start=1):
        for j, chr2 in enumerate(amino2, start=1):
            edit_m[i,j] = local_from_where(distance_mode,direction_m,i,j,
                                     edit_m[i-1,j-1] + d(distance_mode,chr1,chr2),
                                     edit_m[i,j-1] + d(distance_mode,'-',chr2),
                                     edit_m[i-1,j] + d(distance_mode,chr1,'-')) 
    return edit_m, direction_m

In [15]:
def local_from_where(mode,direction_m,i,j,a,b,c):
    if mode == 'distance':
        val = min(a,b,c)
    
    if mode == 'similarity':
        val = max(a,b,c)
    
    if val > 0:
        # from the diagonal
        if val == a:
            direction_m[i,j,0] = 1

        # from the left
        if val == b:
            direction_m[i,j,1] = 1

        # from the upside
        if val == c:
            direction_m[i,j,2] = 1
    else:
        val = 0
        
    return val

In [16]:
# find biggest value and start from there
def local_alignment(direction_m,i,j,str1,str2):  
    if direction_m[i,j,0] == 1:
        str1.insert(0,('-'+amino1)[i])
        str2.insert(0,('-'+amino2)[j])
        i = i - 1
        j = j - 1
        
    elif direction_m[i,j,1] == 1:
        str1.insert(0,'-')
        str2.insert(0,('-'+amino2)[j])
        j = j - 1
        
    elif direction_m[i,j,2] == 1:
        str1.insert(0,('-'+amino1)[i])
        str2.insert(0,'-')
        i = i - 1
        
    print("i:", i, "j:",j)
    
    if (i == 0) or (j == 0):
        return str1, str2
    
    return local_alignment(direction_m,i,j,str1,str2)    

In [17]:
# initialize matrices
local_sim_m = np.zeros((len(amino1)+1,len(amino2)+1), dtype=int)
local_route = np.zeros((len(amino1)+1,len(amino2)+1,3), dtype=int)

In [18]:
local_sim_m, local_route = local_preparation(amino1, amino2, local_sim_m, local_route, 'similarity')

In [19]:
start_i,start_j = np.unravel_index(local_sim_m.argmax(), local_sim_m.shape)

In [20]:
local_alignment(sim_route,start_i,start_j,[],[])

i: 5 j: 2
i: 4 j: 1
i: 3 j: 0


(['c', 'c', 'c'], ['c', 'c', 'c'])